# Install Dependencies

In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests
!pip3 install snscrape


# Import Modules

In [ ]:
from google.colab import files
import snscrape.modules.twitter as sntwitter
import snscrape.modules.telegram as sntelegram
import requests
from bs4 import BeautifulSoup
import time
import re
import json
import string
import pandas as pd
import csv
import math
import os
import tweepy as tw
import random
import datetime
import threading
import glob
import zipfile
import zlib
import multiprocessing
from tweepy import TweepError

In [ ]:
import shutil

shutil.rmtree('/content/sample_data')


# Twitter

In [ ]:
consumer_key= 'PFntvVTEGYxOj0DpsQGLfcHih'
consumer_secret= 'qaMWaTihgx7WKCxs2CWf6VkJrGQMaQyBkd22KW6vhdHpRmO7yg'
access_token= '994786472900333570-6qqX2SH8XbHXAbJRkWdwzge4A7xktPo'
access_token_secret= 'ZryyU4n78wJyUQaNVDPNR7X7a7XKDAOtxy768SCMH2Y2E'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
begin_date = "2015-01-01"
end_date = "2023-02-04"
max_tweets = 100000
counter = 1

In [ ]:
targets = ['_AmranFans']

In [ ]:
for user in targets:
    # Creating list to append tweet data
    tweets_list = []
    id_lst = []
    print(f'Now extracting tweets from {user}, {counter}/{len(targets)}')
    search_key = f'(from:{user}) include:nativeretweets until:{end_date} since:{begin_date}'
    try:
        # Using TwitterSearchScraper to scrape data and append tweets to list
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_key).get_items()):
            if i > max_tweets:
                break
            elif i % 1000 == 0:
                print(f'{i} tweets have been scraped')
            tweets_list.append([tweet.date, tweet.id, tweet.lang, tweet.source,
                                tweet.url, tweet.rawContent, tweet.user.username])
            id_lst.append(tweet.id)
    except:
        print("Protected account!")
        pass
    # Creating a dataframe from the tweets list above
    tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Lang',
                                                   'Source', 'URL', 'Text',
                                                   'Username'])
    if len(tweets_df) > 0:
        tweets_df.to_csv(f'{user.lower()}.csv')
        with open(f'{user.lower()}.json', 'w', encoding='utf-8') as f:
            json.dump(json.dumps(id_lst), f, ensure_ascii=False, indent=4)
    counter = counter + 1

In [ ]:
def searchProfile(userhandle, begin_date=None, end_date=None, max_tweets=None):
    if not isinstance(userhandle, (str)):
        raise ValueError("The argument userhandle must be a string.")
    if not isinstance(begin_date, (str, type(None))):
        raise ValueError("The argument begin_date must be a string.")
    if not isinstance(end_date, (str, type(None))):
        raise ValueError("The argument end_date must be a string.")
    if not isinstance(max_tweets, (int, type(None))):
        raise ValueError("The argument max_tweets must be an Int.")

    if begin_date is None:
        begin_date='2006-03-21'
    if end_date is None:
        end_date = datetime.datetime.today().strftime('%Y-%m-%d')

    print(f'Now extracting tweets from @{userhandle}\n')

    tweets_list = []
    id_lst = []

    try:
        err = 0
        for i, tweet in enumerate(sntwitter.TwitterUserScraper(userhandle).get_items()):
            if max_tweets is not None:
                if i > max_tweets:
                    break

            if i % 1000 == 0 and i != 0:
                print(f'{i} tweets have been scraped from user @{userhandle}\n')

            try:
                tweets_list.append([tweet.date, tweet.id, tweet.lang, tweet.source,
                                    tweet.url, tweet.rawContent, tweet.renderedContent,
                                    tweet.likeCount,
                                    tweet.replyCount, tweet.retweetCount,
                                    tweet.quoteCount,tweet.viewCount,
                                    tweet.hashtags, tweet.inReplyToTweetId,
                                    tweet.inReplyToUser,
                                    tweet.quotedTweet,
                                    tweet.user.username])

                id_lst.append(tweet.id)

            except Exception as e:
                print(e)
                print(f'While extracting tweets from @{userhandle}, an error has occured.')
                err = err + 1
                print(f'While extracting tweets from @{userhandle}, total number of error has increased to {err}')


        print(f"Total of {i - err} tweets have been scraped from @{userhandle}")
        # Creating a dataframe from the tweets list above
        tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Lang',
                                                    'Source', 'URL', 'Text', 'Rendered Content',
                                                    'Like Count',
                                                    'Reply Count', 'Retweet Count', 'Quote Count',
                                                    'View Count', 'Hashtags',
                                                    'inReplyToTweetId',
                                                    'inReplyToUser',
                                                    'quotedTweet',
                                                    'Username'])
        if len(tweets_df) > 0:
            tweets_df.to_csv(f'{tweet.user.username.lower()}.csv')
            with open(f'{tweet.user.username.lower()}.json', 'w', encoding='utf-32') as f:
                json.dump(json.dumps(id_lst), f, ensure_ascii=False, indent=4)
    except:
        print(f"@{userhandle} is a protected account!")
        pass



In [ ]:
def multipleSearchProfile(userhandles, processes_num=1):

    if processes_num <= 0:
        processes_num = len(userhandles)

    with multiprocessing.Pool(processes=processes_num) as pool:
        pool.starmap(searchProfile, [(item,) for item in userhandles])



In [ ]:
filename = r'/content/userhandles.txt'

with open(filename, 'r') as f:
    userhandles = f.read().splitlines()

In [ ]:
multipleSearchProfile(userhandles, processes_num=0)

In [ ]:
!zip -r /content/file.zip /content
files.download(r'/content/')

# Telegram

In [ ]:
telegram_channel_name = 'greatreject'
data = sntelegram.TelegramChannelScraper(telegram_channel_name).get_items()

In [ ]:
i = 0
for d in data:
    if i > 10:
        break
    i = i + 1
    print(f"{d.url} {d.date}\n{d.content}\n{d.outlinks}\n{d.linkPreview}")
    print('-'*8)